<a href="https://colab.research.google.com/github/ptmhoang97/process_dataset/blob/main/convert_VOC_to_YOLOv4_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
from google.colab import drive
drive._mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
# Choose dataset to unzip
dataset_name = "coco2017_train_car_VOC"

# Setup paths unzip dataset from GG drive

In [40]:
import os

In [41]:
dataset_name_VOC = dataset_name

In [42]:
file_VOC_location = "/content/{0}".format(dataset_name_VOC)

In [43]:
dataset_name_YOLOv4 = "{0}_YOLOv4".format(dataset_name[:-4])
dataset_name_YOLOv4_zip = "{0}.zip".format(dataset_name_YOLOv4)

In [44]:
file_YOLOv4_location = "/content/{0}".format(dataset_name_YOLOv4)

In [45]:
paths = {
	'TRAIN_PATH_VOC': os.path.join(file_VOC_location,"train"),
	'TEST_PATH_VOC': os.path.join(file_VOC_location,"test"),
	'TRAIN_PATH_YOLOv4': os.path.join(file_YOLOv4_location,"train"),
	'TEST_PATH_YOLOv4': os.path.join(file_YOLOv4_location,"test"),
}

In [46]:
dataset_zip_path = "/content/gdrive/MyDrive/_dataset/{0}.zip".format(dataset_name)
dataset_unzip_path = "/content"
!unzip {dataset_zip_path} -d {dataset_unzip_path}

Streaming output truncated to the last 5000 lines.
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_5534.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_9072.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_3078.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_3928.xml  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_7556.xml  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_1263.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_2592.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_4284.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_7801.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_5587.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_6660.jpg  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_6377.xml  
  inflating: /content/coco2017_train_car_VOC/train/car_COCO_9623.jpg  
  inflating: /content/coco

# Convert xml to csv

In [47]:
import os
import glob
import pandas as pd
import io
import xml.etree.ElementTree as ET
import argparse

In [48]:
def xml_to_csv(path):
    """Iterates through all .xml files (generated by labelImg) in a given directory and combines
    them in a single Pandas dataframe.

    Parameters:
    ----------
    path : str
        The path containing the .xml files
    Returns
    -------
    Pandas DataFrame
        The produced dataframe
    """

    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text)
                     )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                   'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [49]:
csv_path = "/content/csv"

files_csv = {
	'TRAIN_CSV': os.path.join(csv_path,"train.csv"),
	'TEST_CSV': os.path.join(csv_path,"test.csv"),
}

In [50]:
# Create folder to store generated csv
if not os.path.exists(csv_path):
    !mkdir {csv_path}
else:
    !rm -rf {csv_path}
    !mkdir {csv_path}

In [51]:
train_csv = xml_to_csv(paths['TRAIN_PATH_VOC'])
train_csv.to_csv(files_csv['TRAIN_CSV'], index=None)
print('Successfully created the CSV file: {}'.format(files_csv['TRAIN_CSV']))

Successfully created the CSV file: /content/csv/train.csv


In [52]:
test_csv = xml_to_csv(paths['TEST_PATH_VOC'])
test_csv.to_csv(files_csv['TEST_CSV'], index=None)
print('Successfully created the CSV file: {}'.format(files_csv['TEST_CSV']))

Successfully created the CSV file: /content/csv/test.csv


# Convert csv to xlsx (Keep value and change format)

In [53]:
files_xlsx = {
	'TRAIN_XLSX': os.path.join(csv_path,"train.xlsx"),
	'TEST_XLSX': os.path.join(csv_path,"test.xlsx"),
}

In [54]:
read_file_train_csv = pd.read_csv(files_csv['TRAIN_CSV'])
read_file_train_csv.to_excel(files_xlsx['TRAIN_XLSX'], index = None, header=True)

In [55]:
read_file_test_csv = pd.read_csv(files_csv['TEST_CSV'])
read_file_test_csv.to_excel(files_xlsx['TEST_XLSX'], index = None, header=True)

# Convert xlsx to txt (YOLOv4 annotation)

In [56]:
import shutil

shutil.copytree(file_VOC_location, file_YOLOv4_location)

'/content/coco2017_train_car_YOLOv4'

In [57]:
import openpyxl

In [58]:
def convert_xlsx_to_yolo(input_xlsx,output_yolo):
    wb=openpyxl.load_workbook(input_xlsx)
    sheet = wb['Sheet1']
    #bar = Bar('Processing: ' + input_xlsx, max=sheet.max_row)
    # print(wb.active.title)

    for i in range(2,sheet.max_row+1):
        filename = sheet.cell(i, 1).value[:-4]
        # print(filename)

        width = int(sheet.cell(i, 2).value)
        # print(width)

        height = int(sheet.cell(i, 3).value)
        # print(height)

        class_obj = sheet.cell(i, 4).value
        # print(class_obj)

        x_min = sheet.cell(i, 5).value
        # print(x_min)

        y_min = sheet.cell(i, 6).value
        # print(y_min)

        x_max = sheet.cell(i, 7).value
        # print(x_max)

        y_max = sheet.cell(i, 8).value
        # print(y_max)
        
        output_file_name = "{}/{}.txt".format(output_yolo,filename)
        output_class = "0"
        output_center_x = str((x_max + x_min)/2/width)
        output_center_y = str((y_max + y_min)/2/height)
        output_width = str((x_max - x_min)/width)
        output_height = str((y_max - y_min)/height)

        dark_net_label = os.path.join(output_yolo, "_darknet.labels")

        if not os.path.exists(dark_net_label):
            with open(dark_net_label,"w") as f:
                f.write(class_obj + "\n")

        if not os.path.exists(output_file_name): 
            f = open(output_file_name, "x")
            f.write(output_class + ' ')
            f.write(output_center_x + ' ')
            f.write(output_center_y + ' ')
            f.write(output_width + ' ')
            f.write(output_height + '\n')
            f.close()
        else:
            f = open(output_file_name, "a")
            f.write(output_class + ' ')
            f.write(output_center_x + ' ')
            f.write(output_center_y + ' ')
            f.write(output_width + ' ')
            f.write(output_height + '\n')
            f.close()

In [59]:
convert_xlsx_to_yolo(files_xlsx['TRAIN_XLSX'],paths['TRAIN_PATH_YOLOv4'])

In [60]:
convert_xlsx_to_yolo(files_xlsx['TEST_XLSX'],paths['TEST_PATH_YOLOv4'])

In [61]:
import os
import fnmatch

def delete_all_file(file_location,type_file):
    pattern = '*.{0}'.format(type_file)
    file_path = []
    for path in fnmatch.filter(os.listdir(file_location), pattern):
        full_path = os.path.join(file_location, path)
        if os.path.isfile(full_path):
            file_path.append(full_path)

    if not file_path:
        print("No files with type: " + str(type_file))
    else:
        print("Deleting all files with type: " + str(type_file))
        for path in file_path:
            if path.endswith(type_file):
                os.remove(path)
        print("Done deleting!")

In [62]:
delete_all_file(paths['TRAIN_PATH_YOLOv4'],"xml")

Deleting all files with type: xml
Done deleting!


In [63]:
delete_all_file(paths['TEST_PATH_YOLOv4'],"xml")

Deleting all files with type: xml
Done deleting!


# Zip and copy dataset to GG drive

In [64]:
!zip -r {dataset_name_YOLOv4_zip} {dataset_name_YOLOv4}

Streaming output truncated to the last 5000 lines.
  adding: coco2017_train_car_YOLOv4/train/car_COCO_8320.jpg (deflated 0%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_3809.txt (deflated 58%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_224.txt (deflated 66%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_2279.txt (deflated 52%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_5534.jpg (deflated 0%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_9072.jpg (deflated 0%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_3078.jpg (deflated 5%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_3290.txt (deflated 65%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_1263.jpg (deflated 0%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_2592.jpg (deflated 0%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_4284.jpg (deflated 0%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_2747.txt (deflated 16%)
  adding: coco2017_train_car_YOLOv4/train/car_COCO_7801.j

In [65]:
!cp {dataset_name_YOLOv4_zip} "/content/gdrive/MyDrive/_dataset"